In [1]:
import os, sys
import rasterio as rio
import numpy as np
import geopandas as gpd
import pandas as pd

sys.path.append('/uio/kant/geo-ceed-u1/nilscp/Nils/Python/utils')
import raster

- selection crater
- get centroid
- get old coordinate systems
- project to new coordinate system (equirectangular based on lon, lat) (centroid), I guess I have to do that for 
- apply buffer to the centroid
- feature to envelope to polygon (make a square from the buffered geometry)
- reproject to old coordinate systems
- clip based on the bounding box / geometry
- convert to ascii (but maybe not necessary with this new code)

In [2]:
path = "/run/media/nilscp/pampa/ANALYSIS/SIMPLECRATERS_MOON/SLDEM_2015_LARGE_CRATERS/layers"
filename = os.path.join(path, 'rayed_craters.shp')

In [3]:
# reading the shape file (craters)
df = gpd.read_file(filename)

In [25]:
# because of complicated coordinate systems

default_src_string1 = 'PROJCS["Equirectangular Moon",GEOGCS["GCS_Moon",DATUM["D_Moon",SPHEROID["Moon_localRadius",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433]],PROJECTION["Equirectangular"],PARAMETER["latitude_of_origin",'
                      
default_src_string2 =  '],PARAMETER["central_meridian",0],PARAMETER["standard_parallel_1",'

default_src_string3 = '],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'

In [26]:
#n = np.shape(df)[0]

path_raster = '/uio/kant/geo-ceed-u1/nilscp/Desktop/lunaspec/SLDEM2015'
in_raster = os.path.join(path_raster, 'Lunar_LRO_LrocKaguya_DEMmerge_60N60S_512ppd.tif')
cliptype = 'geojson'
    


for i in range(10,12):
    
    dict_crater = {'geometry': [df.geometry.loc[i]], 'index': [0]}
    dataframe_crater = pd.DataFrame(dict_crater)
    geodataframe_crater = gpd.GeoDataFrame(dataframe_crater, geometry=dataframe_crater.geometry)
    geodataframe_crater.crs = df.crs
    
    geom_geoseries = gpd.GeoSeries(geodataframe_crater.geometry)
    geom_geoseries.crs = df.crs
    
    centroids = geom_geoseries.centroid
    new_crs = df.crs.copy()
    new_crs['lat_0'] = df.y_coord.loc[i]
    new_crs['lon_0'] = df.x_coord.loc[i]
    
    centroids_eq = geom_geoseries.to_crs(new_crs)
    buffer_array = centroids_eq.buffer(df.Diam_km.loc[i]*4000.0)
    envelope_array = buffer_array.envelope
    envelope_global = envelope_array.to_crs(df.crs)
    
    tmp = envelope_global.__geo_interface__ 
    in_polygon = [tmp['features'][0]['geometry']]
    
    # clip
    fname_raster_clip = 'crater' + str(i).zfill(4) + '_clip.tif'
    clipped_raster = os.path.join(path_raster, fname_raster_clip)
    raster.clip(in_raster, in_polygon, cliptype, clipped_raster)
    
    # reproject
    fname_raster_reproj = 'crater' + str(i).zfill(4) + '.tif'
    reproj_raster = os.path.join(path_raster, fname_raster_reproj)
    
    # get new coordinate system (this does not work yet)
    dst_crs_string = (default_src_string1 + str(int(df.y_coord.loc[i])) + default_src_string2 + 
                      str(int(df.x_coord.loc[i])) + default_src_string3)
    
    dst_crs = rio.crs.CRS.from_string(dst_crs_string) #
    
    raster.reproject(clipped_raster, dst_crs, reproj_raster)
    

CRSError: Failed to initialize PROJ pipeline from '+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +a=1737400 +b=1737400 +units=m +no_defs' to '+proj=eqc +lat_ts=-125 +lat_0=13 +lon_0=0 +x_0=0 +y_0=0 +a=1737400 +b=1737400 +units=m +no_defs': lat_ts >= 90

In [27]:
rio.crs.CRS.from_string(dst_crs_string)

CRS.from_wkt('PROJCS["Equirectangular Moon",GEOGCS["GCS_Moon",DATUM["D_Moon",SPHEROID["Moon_localRadius",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433]],PROJECTION["Equirectangular"],PARAMETER["latitude_of_origin",13],PARAMETER["central_meridian",0],PARAMETER["standard_parallel_1",-125],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]')

In [21]:
with rio.open(in_raster) as src:

    default_src = src.crs

In [22]:
default_src.to_string()

'PROJCS["Equirectangular Moon",GEOGCS["GCS_Moon",DATUM["D_Moon",SPHEROID["Moon_localRadius",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433]],PROJECTION["Equirectangular"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",0],PARAMETER["standard_parallel_1",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'

In [19]:
src.crs.from_string(src.crs.to_string())

CRS.from_wkt('PROJCS["Equirectangular Moon",GEOGCS["GCS_Moon",DATUM["D_Moon",SPHEROID["Moon_localRadius",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433]],PROJECTION["Equirectangular"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",0],PARAMETER["standard_parallel_1",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]')

In [8]:
dst_crs

CRS.from_wkt('PROJCS["unnamed",GEOGCS["unnamed ellipse",DATUM["unknown",SPHEROID["unnamed",1737400,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Equirectangular"],PARAMETER["latitude_of_origin",-13.8575981943],PARAMETER["central_meridian",173.464435911],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["Meter",1]]')